In [1]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#query = "anmeldung"
#target_service = 120686

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(10)
eval.head(50)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...
5,328200,20220131--quantEvalAllExcel.xlsx,24,Wo kann ich mich nach Corona Infektion freites...,6,True,Infektionskrankheiten melden - für Gemeinschaf...
6,330073,20220131--quantEvalAllExcel.xlsx,28,Ab wann ist man bei Johnson & Johnson geboostert,5,True,Impfung gegen Corona (SARS-CoV-2)
7,120702,20220131--quantEvalAllExcel.xlsx,36,Genesenen Bescheinigung,6,True,Meldebescheinigung beantragen
8,330150,20220131--quantEvalAllExcel.xlsx,39,"welche medikamente kann man nehmen, wenn man c...",6,True,Arzneimittel - zur Sachkundeprüfung für den Ha...
9,330174,20220131--quantEvalAllExcel.xlsx,47,ab 15.1. ist für Gäste von Restaurants Nachwe...,6,True,Gaststättengewerbe - zum Unterrichtungsnachwei...


In [4]:
eval.iloc[2]["initialQuestion"]

'Pcr test positiv brauche ich eine Krankschreibung vom Arzt für den Arbeitgeber'

Funktionen

In [5]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df#.reset_index()
    rank = temp.index[temp["id"]==str(targetService)].values[0]
    return rank

Logging

In [6]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        
        if answer is not None:

            
            nResults = len(self.chatbot.df.index)
            question = self.chatbot.generateQuestion()
            answer = answer

            rank = getRank(self.chatbot,target_service)# getRank(target_service)
            
            row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
        else:
            #nInitialResults = len(chatbot.df.index)
            #initialRank = np.nan
            row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

Execution

In [7]:
# Initialisierung Komponenten Chatbot
solrhandler = sh.SolrHandler(5)
clusterer = cls.Clusterer()
tpc_dterminator = td.TopicDeterminator()

In [8]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    
    # fun
    query = row["initialQuestion"]
    target_service = row["documentId"]
    
    chatbot = cb.Chatbot(query,3,solrhandler,clusterer,tpc_dterminator)
    lg.setChatbot(chatbot)
    
    # Log
    try:
        chatbot.initialQuery(query)
    except Exception as e:
        if(str(e) == 'response'): # TODO ??
            next
        elif(str(e) == 'no solr output'):
            next
        else:
            print(e)
    
    t = 0
    answer = findCorrectAnswer(chatbot, target_service)
    lg.log(ID,t, row, answer)
    if answer is None:
        continue
    
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):

        # log
        t += 1
        
        answer = findCorrectAnswer(chatbot, target_service)
        chatbot.refineResultset(answer, True)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)

In [ ]:
#query
#len(solrhandler.get_df_from_query(query))

In [9]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "service_name", "query", "question", "answer", "rank", "nResults"])
log_df["rank"] = log_df["rank"]+1
log_df["t"] = log_df["t"] +1
log_df.head()

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults
0,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um ummelden?,True,3.0,5.0
1,20220131--quantEvalAllExcel.xlsx,4,0,2,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um ummelden?,True,2.0,3.0
2,20220131--quantEvalAllExcel.xlsx,4,1,1,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN
3,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um 19?,True,4.0,5.0
4,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um 19?,True,2.0,2.0


Statistics

In [10]:
# avg Rank Improvement in turn t
log_df["rank_improvement"] = log_df.groupby(["ID"])["rank"].diff()
log_df.groupby(["t"])["rank_improvement"].mean()

t
1     NaN
2   -0.75
3    0.00
Name: rank_improvement, dtype: float64

In [11]:
# avg nSuggestions Improvement in turn t
log_df["sugg_improvement"] = log_df.groupby(["ID"])["nResults"].diff()
log_df.groupby(["t"])["sugg_improvement"].mean()

t
1    NaN
2   -2.5
3   -2.0
Name: sugg_improvement, dtype: float64

In [12]:
# MRR ohne cluster refinement
df = log_df.loc[log_df["t"] == 1]
MRR_ohne = 1/(df["rank"].mean())
MRR_ohne

0.4444444444444444

In [13]:
# MRR mit cluster refinement
df = log_df.groupby(["ID"]).last("t")
MRR_mit = 1/(df["rank"].mean())
MRR_mit
#df.head()

0.6666666666666666

In [14]:
# avg Turns (Achtun!) -> chatbot geht noch nicht bis auf einzelnen service herunter weil je nachdem alle in einem cluster
print(log_df["t"].mean())

# bei durchschnittlich nResults
print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

1.4
2.0


In [15]:
log_df.head(50)

,file,dialogId,ID,t,service_name,query,question,answer,rank,nResults,rank_improvement,sugg_improvement
0,20220131--quantEvalAllExcel.xlsx,4,0,1,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um ummelden?,True,3.0,5.0,NaN,NaN
1,20220131--quantEvalAllExcel.xlsx,4,0,2,Änderung/Wechsel der Hauptwohnung,Ummeldung,Geht es bei ihrem Anliegen um ummelden?,True,2.0,3.0,-1.0,-2.0
2,20220131--quantEvalAllExcel.xlsx,4,1,1,Anmeldung einer Wohnung,Ummelden,None,None,NaN,NaN,NaN,NaN
3,20220131--quantEvalAllExcel.xlsx,19,2,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um 19?,True,4.0,5.0,NaN,NaN
4,20220131--quantEvalAllExcel.xlsx,19,2,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,Geht es bei ihrem Anliegen um 19?,True,2.0,2.0,-2.0,-3.0
5,20220131--quantEvalAllExcel.xlsx,19,3,1,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um erstattung?,True,1.0,5.0,NaN,NaN
6,20220131--quantEvalAllExcel.xlsx,19,3,2,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,Geht es bei ihrem Anliegen um erstattung?,True,1.0,2.0,0.0,-3.0
7,20220131--quantEvalAllExcel.xlsx,23,4,1,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,None,None,NaN,NaN,NaN,NaN
8,20220131--quantEvalAllExcel.xlsx,24,5,1,Infektionskrankheiten melden - für Gemeinschaf...,Wo kann ich mich nach Corona Infektion freites...,None,None,NaN,NaN,NaN,NaN
9,20220131--quantEvalAllExcel.xlsx,28,6,1,Impfung gegen Corona (SARS-CoV-2),Ab wann ist man bei Johnson & Johnson geboostert,None,None,NaN,NaN,NaN,NaN
